
# Metadata

**Topic** - Data analysis sql

**Target Number of Turns (User + Assistant) - 2**

**Suggested Conversation :** - Hey!! I am a data analyst at ZephyrTech an online electronics store. We're diving into our sales figures from the last quarter to unearth insights about our customer base to understand the spending patterns of new vs returning customers. Our 'SalesData' table, where transactions are logged, includes 'TransactionID', 'CustomerID', 'CustomerType', 'PurchaseAmount', and 'PurchaseDate'. So I want a sql query for my postgresql db that not only compares the average purchase amounts between these two groups but does so through conditional aggregation. Can you help me devise this query?

**User Personality :** - The user is analytical and engaged, displaying a keen interest in dissecting data patterns. They prefer a practical approach to problem-solving and are confident in their ability to interpret complex SQL queries.


# Conversation

**User**

I need a SQL query for my PostgreSQL database that compares the average purchase amounts between new and returning customers using conditional aggregation on the SalesData table, which includes TransactionID, CustomerID, CustomerType, PurchaseAmount, and PurchaseDate. Can you help me devise this query?

**Assistant**

Certainly! To differentiate between new and returning customers, we'll consider a customer new if their transaction, in question, is their first ever transaction in your `SalesData` table; otherwise, we'll consider them returning.

Here's a SQL query that should meet your requirements:


```sql
-- Calculate the average purchase amount for both new and returning customers
SELECT
  AVG(CASE
        WHEN CustomerFirstPurchaseDate.PurchaseDate = SalesData.PurchaseDate THEN SalesData.PurchaseAmount
      END) AS AveragePurchaseAmountNewCustomers,
  
  AVG(CASE
        WHEN CustomerFirstPurchaseDate.PurchaseDate <> SalesData.PurchaseDate THEN SalesData.PurchaseAmount
      END) AS AveragePurchaseAmountReturningCustomers
FROM
  SalesData
JOIN
  (SELECT CustomerID, MIN(PurchaseDate) AS PurchaseDate
   FROM SalesData
   GROUP BY CustomerID) AS CustomerFirstPurchaseDate
ON
  SalesData.CustomerID = CustomerFirstPurchaseDate.CustomerID;
```

This query does the following:

- Identifies each customer's first purchase using a subquery.

- Joins this data with  `SalesData`  to match each transaction to the first purchase date.

- Calculates average purchase amounts using conditional aggregation:
  -  For new customers, it averages amounts on their first purchase date.
  -  For returning customers, it averages amounts on subsequent dates.

**User**

Could you show how to calculate the median purchase amount for new vs returning customers as well for a more nuanced understanding of spending patterns?

**Assistant**

This can be achieved through creative use of window functions and conditional logic. Given your scenario, first, we separate new vs. returning customers, then calculate the median purchase amount for these two groups. Here’s a way to approach it:


```sql
WITH RankedPurchases AS (
    
    SELECT
        SalesData.CustomerID,
        SalesData.PurchaseAmount,
        -- Determine if the transaction is from a new customer (first purchase)
        CustomerFirstPurchaseDate.PurchaseDate = SalesData.PurchaseDate AS IsNewCustomer,
        ROW_NUMBER() OVER(
            PARTITION BY SalesData.CustomerID, CustomerFirstPurchaseDate.PurchaseDate
            ORDER BY SalesData.PurchaseAmount
        ) AS RowAsc,
        ROW_NUMBER() OVER(
            PARTITION BY SalesData.CustomerID, CustomerFirstPurchaseDate.PurchaseDate
            ORDER BY SalesData.PurchaseAmount DESC
        ) AS RowDesc
    FROM
        SalesData
    JOIN
        -- Subquery to determine each customer's first PurchaseDate
        (SELECT CustomerID, MIN(PurchaseDate) AS PurchaseDate
        FROM SalesData
        GROUP BY CustomerID) AS CustomerFirstPurchaseDate
        ON SalesData.CustomerID = CustomerFirstPurchaseDate.CustomerID
),
PurchaseCounts AS (
  
    SELECT
        IsNewCustomer,
        COUNT(*) AS Total
    FROM RankedPurchases
    GROUP BY IsNewCustomer
),
Medians AS (
   
    SELECT
        RP.IsNewCustomer,
        -- Median computed as the average PurchaseAmount around the center of the dataset
        AVG(RP.PurchaseAmount) AS MedianPurchaseAmount
    FROM RankedPurchases RP
    JOIN PurchaseCounts PC ON RP.IsNewCustomer = PC.IsNewCustomer
   
    WHERE RP.RowAsc BETWEEN PC.Total/2.0 AND PC.Total/2.0 + 1
       OR RP.RowDesc BETWEEN PC.Total/2.0 AND PC.Total/2.0 + 1
    GROUP BY RP.IsNewCustomer
)

SELECT
    CASE
        WHEN IsNewCustomer = 1 THEN 'New Customers'
        ELSE 'Returning Customers'
    END AS CustomerType,
    MedianPurchaseAmount
FROM Medians;
```

The query works as follows:

- Assigns ascending and descending rank to each transaction based on purchase amount, identifying purchases as from new or returning customers.

- Counts transactions for both customer groups.

- Determines the median purchase amount by finding the middle-ranked transactions and averaging if necessary.

- Labels each row as new or returning customer and displays their median purchase amounts.